In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import json
from tqdm import tqdm

tqdm.pandas()

In [ ]:
lib_df = pd.read_csv("data/mce_library_first_metadata_raw.csv")

In [ ]:
lib_df.head()

In [ ]:
# lib_df[["WellLetter", "WellNumber"]] = lib_df["Plate Location"].str.split(expand=True)
# lib_df["WellLetter"] = [value[:1] for value in lib_df["Plate Location"]]
# lib_df["WellNumber"] = [int(value[1:]) for value in lib_df["Plate Location"]]
# lib_df.drop(columns=["WellNumber", "WellLetter"], inplace=True)
lib_df[["WellNumber", "WellLetter"]] = [[int(value[1:]),value[:1]] for value in lib_df["Plate Location"]]

In [ ]:
sorted_df = lib_df.sort_values(by=["RackCode", "WellLetter", "WellNumber"], ascending=[True, True, True])
sorted_df

In [ ]:
def next_well(well, letter, plate):
    well += 1
    if well > 24:
        well = 1

        # next letter
        letter = ord(letter) + 1
        if letter>80:
            letter = 65
            plate += 1

        letter = chr(letter)

    return well, letter, plate


plate = 1
well = 1
letter = "A"
original_letter = "A"

mixed_plates = []
mixed_wells = []

for index, row in sorted_df.iterrows():
    tmp_letter = row["WellLetter"]
    if tmp_letter is not original_letter:
        original_letter = tmp_letter
        well, letter, plate = next_well(well, letter, plate)

    mixed_plates.append(plate)
    mixed_wells.append("{}{}".format(letter, well))

sorted_df["MixedPlate"] = mixed_plates
sorted_df["MixedWell"] = mixed_wells
sorted_df["mixed_location_plate1"] = ["1D{}_{}".format(plate, well) for well, plate in zip(mixed_wells, mixed_plates)]
sorted_df["mixed_location_plate2"] = ["2D{}_{}".format(plate, well) for well, plate in zip(mixed_wells, mixed_plates)]
sorted_df["mixed_location_plate3"] = ["3D{}_{}".format(plate, well) for well, plate in zip(mixed_wells, mixed_plates)]
sorted_df

In [ ]:
sorted_df.to_csv("data/mce_library.csv", sep="\t", index=False)